In [1]:
import numpy as np

BPF_EEG_data = np.load('_NPY/BPF_EEG_data.npy')
CAR_EEG_data = np.load('_NPY/CAR_EEG_data.npy')
CSP_Feature_data = np.load('_NPY/CSP_Feature_data.npy')
CSP_Filter = np.load('_NPY/CSP_Filter.npy')
int_labels = np.load('_NPY/int_labels.npy')
ori_Epoch_data = np.load('_NPY/ori_Epoch_data.npy')
ori_Labels = np.load('_NPY/ori_Labels.npy')

In [2]:
import pywt
import numpy as np

def extract_wavelet_features(data, wavelet='db4'):
    features = []
    for signal in data:
        coeffs = pywt.wavedec(signal, wavelet, level=5)
        features.append([np.mean(np.abs(coeff)) for coeff in coeffs] + [np.std(coeff) for coeff in coeffs])
    return np.array(features)

# 웨이블릿 특징 추출
wavelet_features = extract_wavelet_features(ori_Epoch_data)

In [3]:
def extract_fft_features(data):
    fft_features = []
    for signal in data:
        fft_coeffs = np.fft.rfft(signal, axis=1)
        amplitude = np.abs(fft_coeffs)
        phase = np.angle(fft_coeffs)
        fft_features.append(np.concatenate([amplitude.flatten(), phase.flatten()]))
    return np.array(fft_features)

# FFT 특징 추출
fft_features = extract_fft_features(ori_Epoch_data)

In [4]:
from scipy.signal import welch

def extract_psd_features(data):
    psd_features = []
    for signal in data:
        freqs, power = welch(signal, fs=256, nperseg=256)
        psd_features.append(power.flatten())
    return np.array(psd_features)

# PSD 특징 추출
psd_features = extract_psd_features(ori_Epoch_data)

In [5]:
# 특징들을 하나의 배열로 결합
combined_features = np.concatenate([wavelet_features, fft_features, psd_features], axis=1)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(combined_features, int_labels, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC(kernel='linear', random_state=42))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the classifier on the test set: {accuracy:.4f}")

Accuracy of the classifier on the test set: 0.0559
